In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import random
import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from fastmri_recon.data.sequences.oasis_sequences import Masked2DSequence, ZeroFilled2DSequence
from fastmri_recon.evaluate.metrics.np_metrics import METRIC_FUNCS, Metrics
from fastmri_recon.evaluate.reconstruction.zero_filled_reconstruction import reco_and_gt_zfilled_from_val_file
from fastmri_recon.evaluate.reconstruction.cross_domain_reconstruction import reco_and_gt_net_from_val_file
from fastmri_recon.evaluate.reconstruction.unet_reconstruction import reco_and_gt_unet_from_val_file 
from fastmri_recon.models.functional_models.cascading import cascade_net
from fastmri_recon.models.functional_models.kiki import kiki_net
from fastmri_recon.models.functional_models.kiki_sep import full_kiki_net
from fastmri_recon.models.functional_models.pdnet import pdnet
from fastmri_recon.models.functional_models.unet import unet
from fastmri_recon.models.utils.non_linearities import lrelu

In [2]:
np.random.seed(0)

In [3]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [4]:
AF = 4

In [5]:
# paths
train_path = '/media/Zaccharie/UHRes/OASIS_data/'
train_gen = Masked2DSequence(
    train_path,
    af=AF,
    inner_slices=32,
    scale_factor=1e-2,
    seed=0,
    rand=True,
    val_split=0.1,
)
val_gen_mask = train_gen.val_sequence
n_train = 1000
n_val = 2
random.seed(0)
train_gen.filenames = random.sample(train_gen.filenames, n_train)
val_gen_mask.filenames = random.sample(val_gen_mask.filenames, n_val)

train_gen_zero = ZeroFilled2DSequence(
    train_path,
    af=AF,
    inner_slices=32,
    scale_factor=1e-2,
    seed=0,
    rand=False,
    val_split=0.1,
    n_pooling=3,
)
val_gen_zero = train_gen_zero.val_sequence
random.seed(0)
train_gen_zero.filenames = random.sample(train_gen_zero.filenames, n_train)
val_gen_zero.filenames = random.sample(val_gen_zero.filenames, n_val)

In [6]:
all_net_params = [
    {
        'name': 'unet',
        'init_function': unet,
        'run_params': {
            'n_layers': 4,
            'pool': 'max',
            "layers_n_channels": [16, 32, 64, 128],
            'layers_n_non_lins': 2,
        },
        'val_gen': val_gen_zero,
        'run_id': 'unet_af4_oasis_1570619888',
    },
    {
        'name': 'pdnet',
        'init_function': pdnet,
        'run_params': {
            'n_primal': 5,
            'n_dual': 5,
            'n_iter': 10,
            'n_filters': 32,
        },
        'run_id': 'pdnet_af4_oasis_1570722239',
    },
    {
        'name': 'cascadenet',
        'init_function': cascade_net,
        'run_params': {
            'n_cascade': 5,
            'n_convs': 5,
            'n_filters': 48,
            'noiseless': True,
        },
        'run_id': 'cascadenet_af4_oasis_1569491836',
    },
    {
        'name': 'kikinet-sep',
        'init_function': full_kiki_net,
        'run_params': {
            'n_convs': 16,
            'n_filters': 48,
            'noiseless': True,
            'activation': lrelu,
        },
        'run_id': 'kikinet_sep_I2_af4_oasis_1572552792',
        'epoch': 50,
    },
]

In [7]:
def unpack_model(init_function=None, run_params=None, run_id=None, epoch=300, **dummy_kwargs):
    try:
        model = init_function(input_size=(None, None, 1), fastmri=False, **run_params)
    except:
        model = init_function(input_size=(None, None, 1), **run_params)
    chkpt_path = f'../checkpoints/{run_id}-{epoch}.hdf5'
    model.load_weights(chkpt_path)
    return model

def metrics_for_params(val_gen=None, name=None, **net_params):
    if val_gen is None:
        val_gen = val_gen_mask
    model = unpack_model(**net_params)
    metrics = Metrics(METRIC_FUNCS)
    pred_and_gt = [
        reco_and_gt_net_from_val_file(*val_gen[i], model)
        for i in tqdm_notebook(range(len(val_gen)), desc=f'Val files for {name}')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc=f'Stats for {name}'):
        metrics.push(images, im_recos)
    return metrics


def metrics_zfilled():
    metrics = Metrics(METRIC_FUNCS)
    pred_and_gt = [
        reco_and_gt_zfilled_from_val_file(*val_gen_mask[i], crop=False)
        for i in tqdm_notebook(range(len(val_gen_mask)), desc='Val files for z-filled')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Stats for z-filled'):
        metrics.push(images, im_recos)
    return metrics

In [8]:
%%time
metrics = []
for net_params in all_net_params:
    metrics.append((net_params['name'], metrics_for_params(**net_params)))
    
metrics.append(('zfilled', metrics_zfilled()))


CPU times: user 31.7 s, sys: 6.6 s, total: 38.3 s
Wall time: 35.4 s


In [9]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [10]:
metrics

[('unet', PSNR = 31.21 +/- 6.114 SSIM = 0.8816 +/- 0.08118),
 ('pdnet', PSNR = 33.84 +/- 7.215 SSIM = 0.9254 +/- 0.06587),
 ('cascadenet', PSNR = 33.28 +/- 3.272 SSIM = 0.9176 +/- 0.01759),
 ('kikinet-sep', PSNR = 31.33 +/- 3.639 SSIM = 0.8821 +/- 0.03188),
 ('zfilled', PSNR = 27.43 +/- 3.775 SSIM = 0.6885 +/- 0.05872)]

In [11]:
metrics.sort(key=lambda x: x[1].metrics['PSNR'].mean())

In [12]:
def n_model_params_for_params(reco_function=None, val_gen=None, name=None, **net_params):
    model = unpack_model(**net_params)
    n_params = model.count_params()
    return n_params

In [13]:
%%time
n_params = {}
for net_params in all_net_params:
    n_params[net_params['name']] =  n_model_params_for_params(**net_params)
    
n_params['zfilled'] =  0

CPU times: user 4.33 s, sys: 94.9 ms, total: 4.43 s
Wall time: 4.12 s


In [14]:
def runtime_for_params(val_gen=None, name=None, **net_params):
    if val_gen is None:
        val_gen = val_gen_mask
    model = unpack_model(**net_params)
    data = val_gen[0]
    start = time.time()
    reco_and_gt_net_from_val_file(*data, model)
    end = time.time()
    return end - start

def runtime_zfilled():
    data = val_gen_mask[0]
    start = time.time()
    reco_and_gt_zfilled_from_val_file(*data, crop=False)
    end = time.time()
    return end - start

In [15]:
%%time
runtimes = {}
for net_params in tqdm_notebook(all_net_params):
    runtimes[net_params['name']] =  runtime_for_params(**net_params)
    
runtimes['zfilled'] = runtime_zfilled()


CPU times: user 14.7 s, sys: 2.13 s, total: 16.8 s
Wall time: 15.3 s


In [16]:
metrics_table = pd.DataFrame(
    index=[name for name, _ in metrics], 
    columns=['PSNR-mean (std) (dB)', 'SSIM-mean (std)', '# params', 'Runtime (s)'],
)
for name, m in metrics:
    metrics_table.loc[name, 'PSNR-mean (std) (dB)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['PSNR'].mean(), 
        std=m.metrics['PSNR'].stddev(),
    )
    metrics_table.loc[name, 'SSIM-mean (std)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['SSIM'].mean(), 
        std=m.metrics['SSIM'].stddev(),
    )
    metrics_table.loc[name, '# params'] = "{}".format(
        n_params[name], 
     )
    metrics_table.loc[name, 'Runtime (s)'] = "{runtime:.4}".format(
        runtime=runtimes[name], 
    )

In [17]:
metrics_table

,PSNR-mean (std) (dB),SSIM-mean (std),# params,Runtime (s)
zfilled,27.43 (1.888),0.6885 (0.02936),0,0.113
unet,31.21 (3.057),0.8816 (0.04059),481801,0.2403
kikinet-sep,31.33 (1.819),0.8821 (0.01594),1251080,2.422
cascadenet,33.28 (1.636),0.9176 (0.008793),424570,0.9736
pdnet,33.84 (3.608),0.9254 (0.03293),318280,5.262
